<a href="https://colab.research.google.com/github/ANUPRIYAR/Sentiment_Analysis_of_IMDB_MovieReviews/blob/main/imdb_movie_review_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip archive.zip

Archive:  archive.zip
  inflating: IMDB Dataset.csv        


In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
path = '/content/IMDB Dataset.csv'
df = pd.read_csv(path)

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
# Map positive to 1 and negaitve to 0 in the Dataframe
df['sentiment'] = df.sentiment.map({'positive': 1 , 'negative' : 0})

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
# We create a new column called kfold
df['kfold'] = -1

In [9]:
# Randomize the rows of data
df = df.sample(frac=1).reset_index(drop=True)

In [10]:
# Fetch Labels
y = df.sentiment.values

In [11]:
# Initiate the kfold class from model_slection module
kf = model_selection.StratifiedKFold(n_splits=5)

In [12]:
# fill the new kfold col
for f, (t_,v_) in enumerate(kf.split(X=df, y=y)):
  df.loc[v_,'kfold'] = f


In [13]:
df.head()

,review,sentiment,kfold
0,Non-stop action and just about every conceivab...,1,0
1,Okay...so I am gazing through my Mom and Dad's...,1,0
2,Apart from the usual stereotypes of the thirti...,1,0
3,"This is a straight-to-video movie, so it shoul...",1,0
4,The definition of a vampire is an inhumane cor...,1,0


In [14]:
df['kfold'].value_counts()

4    10000
3    10000
2    10000
1    10000
0    10000
Name: kfold, dtype: int64

In [16]:
%%time
# we go over the fold created
for fold_ in range(5):
  # temprory dataframes for train and test
  train_df = df[df.kfold != fold_].reset_index(drop=True)
  test_df = df[df['kfold'] == fold_].reset_index(drop=True)

  # Initialize TfidfVectorizer
  tfidf_vec = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

  # fit on train data
  tfidf_vec.fit(train_df.review)

  # Transform train and test data
  xtrain = tfidf_vec.transform(train_df.review)
  xtest = tfidf_vec.transform(test_df.review)

  # Initialize Logistic Regression model
  model  = linear_model.LogisticRegression()
  model.fit(xtrain,train_df.sentiment)


  # Make predictions on test data reviews
# Threshold for prediction is 0.5
  preds = model.predict(xtest)

  accuracy = metrics.accuracy_score(test_df.sentiment, preds)
  print(f"Fold: {fold_}")
  print(f"Accuracy: {accuracy}")

Fold: 0
Accuracy: 0.8964
Fold: 1
Accuracy: 0.9011
Fold: 2
Accuracy: 0.8992
Fold: 3
Accuracy: 0.9001
Fold: 4
Accuracy: 0.8911
CPU times: user 17min 5s, sys: 22.3 s, total: 17min 27s
Wall time: 17min 3s
